In [1]:
import requests , time, random, traceback
from bs4 import BeautifulSoup as bs
import pandas as pd

In [38]:
def getHiddenParams(url):

    # 取得參數

    res0 = requests.get(url)
    soup0 = bs(res0.text,"lxml")

    cookie = soup0.select("input[name='cookietime']")[0]["value"]
    formhash = soup0.select("input[name='formhash']")[0]["value"]
#     print("取得隱藏參數-","cookie:", cookie, "/formhash:", formhash)
    return {"cookie": cookie, "formhash": formhash}

#############################################

def login(cookie, formhash):    
    # 登入
    login_url = "https://www.eyny.com/member.php?mod=logging&action=login&loginsubmit=yes&loginhash=LZbj0&inajax=1"
    login_headers = {
        "Host"      : "www.eyny.com",
        "Origin"    : "https://www.eyny.com",
        "Referer"   : "https://www.eyny.com/member.php?mod=logging&action=login",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
    }

    payload = {
        "formhash"            : formhash,
        "referer"             : "https://www.eyny.com/./",
        "loginfield"          : "username",
        "username"            : "7003un",
        "password"            : "0939771419",
        "questionid"          : "0",
        "answer"              : "",
        "cookietime"          : cookie,
        "g-recaptcha-response": "03AOLTBLTUd1qbWQ38a7xd157mbTo1CKjAR4-X9S3tZdgdNuhZ3pIlWJRhJYc_ozrF38X3DByrbdGjdVr0YnFtL9MOIITlVoKBwsPbsD8bzklK3c5-FBSg9mgJa3JOG19kVnXsyNmSSXl7y8Vt57bzNjtzNohp5pH1YNtdFuqlDOQtIEX4_b3hDvHMPt56bOdd1Xd85FyaLHlconYo4Aia6ZpUGkAr7T2fBSi7Ix8crMUllEqQ2MaIWIUOPxklULMunzIXR2zq8W2rnzdRuwbtEgLbfZqhIq8956uQg1En3xJobMS6RK_0cWGyIoMesNs8351fTSn1i_ZwN3ujwIotX5VymQGafR4GWWqgzFdQAL8PBheU8J0NFBvm6cCbXAdvPeFl0sBXyOjO"
    }

    sess = requests.session()
    res1 = sess.post(login_url, headers = login_headers, data = payload)
    
    return sess



def crawlArticleList(sess):
    # 取得1-10頁的連結
    base_url = "https://www.eyny.com/"
    headers={
        "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
    }

    D3 = base_url+"forum-1710-1.html"

    resD3 = sess.get(D3, headers = headers)
    soupD3 = bs(resD3.text,"lxml")
    
    page_info = soupD3.select("div.pg a")

    page_link_list = [ base_url + ele["href"] for ele in page_info[1:10] ]
    page_link_list.insert(0,D3)

#     print(len(page_link_list))

    # 取得所有文章列表 1-10頁的所有文章連結
    all_page_article_links = []
    for page in page_link_list:

        headers={
            "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
        }

        respages = loginsess.get(page, headers = headers)
        soupD3page = bs(respages.text,"lxml")

        if page == "https://www.eyny.com/forum-1710-1.html":
            for ele in soupD3page.select("div#threadlist th.common a.xst")[3:]:
                all_page_article_links.append(base_url + ele["href"] )
        else:
            for ele in soupD3page.select("div#threadlist th.common a.xst"):
                all_page_article_links.append(base_url + ele["href"] )
    #     [ all_page_article_link.append(base_url + ele["href"] for ele in soupD3page.select("div#threadlist th.common a.xst"))]
    # 為何報錯    
#         print(page, "is done")

    return all_page_article_links 



#############################################

#取得文章資訊

def getArticleData(sess,link):

    headers={
            "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
        }
    res_article = sess.get(link, headers = headers)
    soup_article = bs(res_article.text,"lxml")

    divTable = soup_article.select("div#postlist > div")

    
    
    reply_data_list = []
    for i in divTable[1:-1]:
        replyDetail = replyInfo(i)
        
        reply_data_list.append(replyDetail)
        
           
    
    author = getAuthorInfo(soup_article)
    content = getContent(soup_article)
    articleData ={
        "title"               : content["title"],
        "time"                : content["time"],
        "author"              : author["author"],
        "review"              : content["review"],
        "reply number"        : content["res_cnt"],
        "author post"         : author["author_post_cnt"],
        "author score"        : author["author_score"],
        "author diving value" : author["author_diving_value"],
        "Content"             : content["content"],
        "reply"               : reply_data_list

    }
    return articleData

########################
#存擋
def saveFile(dataList,name):

    df = pd.DataFrame(dataList)
    df = df[["title","time","reply number","review","Content","author","author post","author score","author diving value","reply"]]
    df.to_excel(name,index=False)



In [37]:
#############################################

#取得作者資訊
def getAuthorInfo(soup_article):


    divTable = soup_article.select("div#postlist > div")
    # 作者資訊
    author = soup_article.select("div.authi")[0].text.replace("\n","")
    author_post_cnt = divTable[0].select("dl.pil dd")[0].text
    author_score = divTable[0].select("dl.pil dd")[1].text.replace(" 點","")
    author_diving_value = divTable[0].select("dl.pil dd")[2].text.replace(" 米","")
    return {"author": author, 
            "author_post_cnt": author_post_cnt, 
            "author_score": author_score, 
            "author_diving_value": author_diving_value
           }
#######################################
def getContent(soup_article):
    
    divTable = soup_article.select("div#postlist > div")
    content = divTable[0].select("tr td.t_f")[0].text.replace("\r\n","\n").replace("\n"," ")
    
    #貼文資訊     
    title = soup_article.select("div#postlist h1 a")[0].text + soup_article.select("div#postlist h1 a")[1].text
    time = divTable[0].select("div.pi em")[0].text.replace("發表於 ","")
    review = soup_article.select("div#postlist span.xi1")[0].text
    res_cnt = soup_article.select("div#postlist span.xi1")[1].text
    content = divTable[0].select("tr td.t_f")[0].text.replace("\r\n","\n").replace("\n"," ")
    
    return {"content": content, 
            "title": title, 
            "review": review, 
            "time": time,
            "res_cnt":res_cnt
           }

#######################################

####取得回覆者資訊
def replyInfo(divTable):
    
    reply_author = divTable.select("div.authi")[0].text.replace("\n","")
    if "發表於" in reply_author:
        reply_author = "該用戶已被刪除"
        reply_content = divTable.select("tr td.t_f")[0].text.replace("\r\n","\n").replace("\n"," ")
        reply_post_cnt = "-"
        reply_score = "-"
        reply_diving_value = "-"
        reply_time = divTable.select("div.authi")[0].text.replace("\n","").replace("發表於 ","").replace("|只看該作者","")
        
        data={
        "replyer"               : reply_author,
        "reply time"            : reply_time,
        "replyer's post"        : reply_post_cnt,
        "replyer's score"       : reply_score,
        "replyer's diving value": reply_diving_value,
        "reply content"         : reply_content           
        }
    else:

        reply_content = divTable.select("tr td.t_f")[0].text.replace("\r\n","\n").replace("\n"," ")
        reply_post_cnt = divTable.select("dl.pil dd")[0].text
        reply_score = divTable.select("dl.pil dd")[1].text.replace(" 點","")
        reply_diving_value = divTable.select("dl.pil dd")[2].text.replace(" 米","")
        reply_time = divTable.select("div.pi em")[0].text
        
        data={
        "replyer"               : reply_author,
        "reply time"            : reply_time,
        "replyer's post"        : reply_post_cnt,
        "replyer's score"       : reply_score,
        "replyer's diving value": reply_diving_value,
        "reply content"         : reply_content           
        }
 
    return data




In [46]:
# step1 取得變數
url = "https://www.eyny.com/member.php?mod=logging&action=login"

hiddenParams = getHiddenParams(url)
print("Step1 取得參數 OK")

# step2 登入
loginsess = login( hiddenParams["cookie"], hiddenParams["formhash"])
print("Step2 登入 OK")


# step3 文章列表
articleList = crawlArticleList(loginsess)
print("step3 文章列表取得 OK")

# step4 取得文章作者資訊
article_list = []
for link in articleList[1:10]:
    try:
        articleInfo = getArticleData(loginsess,link)
        article_list.append(articleInfo)
        print(link, "done")
    except:
        traceback.print_exc()
        print(link, "fail")
print(article_list)

# step5 存檔
saveFile(article_list, "20190820-1-D3-EVNY.xlsx")

Step1 取得參數 OK
Step2 登入 OK
step3 文章列表取得 OK
https://www.eyny.com/thread-12311293-1-GU7Y04C7.html done
https://www.eyny.com/thread-12290829-1-GU7Y04C7.html done
https://www.eyny.com/thread-9317174-1-GU7Y04C7.html done
https://www.eyny.com/thread-12309743-1-GU7Y04C7.html done
https://www.eyny.com/thread-11774863-1-GU7Y04C7.html done
https://www.eyny.com/thread-12229666-1-GU7Y04C7.html done
https://www.eyny.com/thread-12252246-1-GU7Y04C7.html done
https://www.eyny.com/thread-12255492-1-GU7Y04C7.html done
https://www.eyny.com/thread-11311869-1-GU7Y04C7.html done
[{'title': '[綜合討論]將近一年沒進遊戲', 'time': '3\xa0小時前', 'author': 'kyo111111', 'review': '17', 'reply number': '0', 'author post': '728', 'author score': '1078', 'author diving value': '34365', 'Content': '今天放假進去打個幾場 發現多重獵人 可以輕鬆過100了 之前打得很吃力才過 現在輕鬆過了 可能又改了不少\xa0\xa0我裝備也都沒變動 ', 'reply': []}, {'title': '[綜合討論]賽季18會不少傳奇會增加特效, 大家要回鍋嗎?', 'time': '2019-8-3 08:22 PM', 'author': 'atpx2', 'review': '2173', 'reply number': '32', 'author post': '1904'

In [44]:
article_list

[{'title': '[綜合討論]將近一年沒進遊戲',
  'time': '3\xa0小時前',
  'author': 'kyo111111',
  'review': '17',
  'reply number': '0',
  'author post': '728',
  'author score': '1078',
  'author diving value': '34365',
  'Content': '今天放假進去打個幾場 發現多重獵人 可以輕鬆過100了 之前打得很吃力才過 現在輕鬆過了 可能又改了不少\xa0\xa0我裝備也都沒變動 ',
  'reply': []},
 {'title': '[綜合討論]賽季18會不少傳奇會增加特效, 大家要回鍋嗎?',
  'time': '2019-8-3 08:22 PM',
  'author': 'atpx2',
  'review': '2173',
  'reply number': '32',
  'author post': '1904',
  'author score': '1135',
  'author diving value': '29490',
  'Content': '賽季18會不少傳奇會增加特效, 大家要回鍋嗎? 目前看藍帖舊的沒傳奇特效部分會加上去 共13個新傳奇能力 目前看到以下的訊息 逝水年華: 秘壇時間加倍  劊子手: 斬殺生命值剩下(5-10%) 的敵人  岩石護手: 遭受攻擊會逐漸將你變成石頭 \xa0 \xa0\xa0 \xa0\xa0 \xa0 (增加50%護甲，但移動速度減少15%，攻速減少20%，可堆疊5次)  迴盪狂怒: 斬殺敵人將讓揮舞者發狂 \xa0 \xa0\xa0 \xa0\xa0 \xa0 (每殺死一個敵人增加攻速15%，移動速度5%，可堆疊5次)  阿扭的項鍊: 若你當下沒有受到傷害，你造成的傷害增加100%，受到的傷害增加50%  梅希斯密特之劫掠者: 每殺死1個敵人減少你其中一樣技能的冷卻時間1秒  莫提克護腕: 狂戰之怒獲得所有符文效果   惡怨(鐵匠打的咒物): 巨靈惡屍以外的巨屍獲得順劈斬和冰寒效果 (!!)   感覺有回鍋的價值?   ',
  'reply': [{'replyer': '1237689',
    '

In [45]:
df = pd.DataFrame(article_list)
df

,title,time,author,review,reply number,author post,author score,author diving value,Content,reply
0,[綜合討論]將近一年沒進遊戲,3 小時前,kyo111111,17,0,728,1078,34365,今天放假進去打個幾場 發現多重獵人 可以輕鬆過100了 之前打得很吃力才過 現在輕鬆過了 可...,[]
1,"[綜合討論]賽季18會不少傳奇會增加特效, 大家要回鍋嗎?",2019-8-3 08:22 PM,atpx2,2173,32,1904,1135,29490,"賽季18會不少傳奇會增加特效, 大家要回鍋嗎? 目前看藍帖舊的沒傳奇特效部分會加上去 共13...","[{'replyer': '1237689', 'reply time': '發表於 201..."
2,[綜合討論]我竟然打到了SOJ!!!!,2014-1-5 02:12 PM,lishengchang,18481,169,27,153,4160,天呀~~我等了10多年，從D2打到D3，終於打到人生第一顆SOJ啦!!!! 2014-1...,"[{'replyer': '0913695966', 'reply time': '發表於 ..."
3,[玩家互動]第18季Nintendo Switch D3 LINE群組收人囉～美版～如有違規請刪除,昨天 04:47 AM,q11231129,145,0,17,148,16071,本帖最後由 q11231129 於 2019-8-20 12:07 AM 編輯 ...,[]
4,[抱怨專區]有人無緣無故被鎖帳號嗎?,2018-5-19 06:14 PM,mtf931,5025,16,22,43,1040,我弟去網咖玩.然後就被鎖帳號了.,"[{'replyer': '鍾小剛', 'reply time': '發表於 2018-5-..."
5,[綜合討論]讓遊戲更好玩的方式,2019-6-11 09:59 PM,herbert111,3782,20,19,0,1190,目前遊戲內大部分的傳奇裝都是廢物，好用好打的配置每個職業就是那2~3種， 每種配裝都是固定的...,"[{'replyer': 'hiphopleg', 'reply time': '發表於 2..."
6,[綜合討論]hinet這幾天連亞服超卡,2019-7-2 01:03 AM,milhouse1112,752,1,669,-2,23820,如題 我是hinet光世代100M 這陣子連亞洲一進遊戲4XXping 慢慢的會降下來剩2x...,"[{'replyer': 'yunyumei', 'reply time': '發表於 前天..."
7,[綜合討論]手機板的暗黑,2019-7-4 08:43 PM,a4726007,2284,13,12,15,250,我在巴哈有看到這方面的訊息 誰知道大概哪時候會出阿 有沒有內幕,"[{'replyer': 'angelloveangel0', 'reply time': ..."
8,[綜合討論]分享打到的洪荒,2017-4-9 11:28 PM,ccf22222,49865,254,320,6,22278,剛剛 上傳下載附件 (1.19 MB) 剛剛 上傳下載附件 (1.29 MB)...,"[{'replyer': '最終守護者', 'reply time': '發表於 2017-..."
